# Example Notebook for blogpost 3 Tips for First-Time Machine Learning Projects


## 1. Shared Data Folder in Colab
This code will only work if the notebook is executed inside Google Colab and the file: 'Station-Inventory.csv.gz' has been added inside a 'Shared-Data' folder.

In [2]:
from google.colab import drive
import pandas as pd

# Make Google Drive available inside noteboook
drive.mount('/content/gdrive')

# Path containing all data used in the project
root_path = '/content/gdrive/My Drive/Shared-Data/'

# Path of specific notebook needed for this notebook
path = root_path + 'Station-Inventory.csv.gz'

# Read in gzip file as pandas dataframe
weather_stations = pd.read_csv(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
len(weather_stations)

8766

## 2. aiohttp to speed up data acquisition

In [5]:
!pip install aiohttp

     |████████████████████████████████| 1.2MB 1.7MB/s 
     |████████████████████████████████| 266kB 10.5MB/s 
     |████████████████████████████████| 153kB 10.8MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=eb443633b98c5ce97b4aff6dec72f13c3425c2c354f56e51c902c80a2d214a4f
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [6]:
import io
import time
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()

In [7]:
async def fetch_dataframe(session, url):
  """ Asynchronously fetches a csv file using the url """
  async with session.get(url) as response:
    byte_code = await response.content.read()
    df = pd.read_csv(io.BytesIO(byte_code), encoding='utf8')
    return df
      
async def fetch_concurrent(urls):
  """ Make concurrent calls """
  loop = asyncio.get_event_loop()
  async with aiohttp.ClientSession() as session:
    tasks = []
    for url in urls:
      tasks.append(loop.create_task(fetch_dataframe(session, url)))

    all_dfs = []

    # Loop through each completed task
    for df in asyncio.as_completed(tasks):
      df = await df
      all_dfs.append(df)

    # Concatenate all fetched dataframes into one
    return pd.concat(all_dfs, ignore_index=True)

In [8]:
def get_url(stationID, year, month):
    """ Gets url for downloading weather csv """
    url = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&" +\
          f"stationID={stationID}&Year={year}&Month={month}&Day=14&timeframe=1" +\
          "&submit=Download+Data"
    return url

In [9]:
tic = time.time()
urls = [get_url(stationID=31468, year=year, month=month) for month in range(1,13) for year in range(2010, 2019)]
df = asyncio.run(fetch_concurrent(urls))
toc = time.time()
print(toc-tic)
df.head()

38.51241493225098


,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),Temp Flag,Dew Point Temp (°C),Dew Point Temp Flag,Rel Hum (%),Rel Hum Flag,Wind Dir (10s deg),Wind Dir Flag,Wind Spd (km/h),Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-118.29,55.62,PEORIA AGDM,3075160,2014-12-01 00:00,2014,12,1,00:00,-25.6,NaN,-28.2,NaN,79.0,NaN,8.0,NaN,4.0,NaN,NaN,NaN,NaN,M,NaN,NaN,-30.0,NaN,NaN
1,-118.29,55.62,PEORIA AGDM,3075160,2014-12-01 01:00,2014,12,1,01:00,-26.9,NaN,-29.7,NaN,77.0,NaN,6.0,NaN,3.0,NaN,NaN,NaN,NaN,M,NaN,NaN,-30.0,NaN,NaN
2,-118.29,55.62,PEORIA AGDM,3075160,2014-12-01 02:00,2014,12,1,02:00,-26.4,NaN,-29.2,NaN,78.0,NaN,1.0,NaN,6.0,NaN,NaN,NaN,NaN,M,NaN,NaN,-32.0,NaN,NaN
3,-118.29,55.62,PEORIA AGDM,3075160,2014-12-01 03:00,2014,12,1,03:00,-27.1,NaN,-30.2,NaN,75.0,NaN,33.0,NaN,3.0,NaN,NaN,NaN,NaN,M,NaN,NaN,-30.0,NaN,NaN
4,-118.29,55.62,PEORIA AGDM,3075160,2014-12-01 04:00,2014,12,1,04:00,-25.7,NaN,-28.3,NaN,79.0,NaN,30.0,NaN,5.0,NaN,NaN,NaN,NaN,M,NaN,NaN,-31.0,NaN,NaN


In [10]:
len(df)

78888

## 3. Vectorizing dataframe operations

Formula for converting relating humidity to absolute humiditiy: https://carnotcycle.wordpress.com/2012/08/04/how-to-convert-relative-humidity-to-absolute-humidity/

In [19]:
import math

def calculate_abs_humidity(temperature, relative_humidity):
  """ Calculates absolute humidity from temperature and relative humidity """
  t = temperature
  rh = relative_humidity
  abs_humidity = (6.112 * math.exp((17.67*t)/(t+243.5)) * rh * 2.1674)/(273.15+t)
  return abs_humidity

In [20]:
%%timeit -n 3
abs_humidity_list = []
for index, row in df.iterrows():
    temperature = row['Temp (°C)']
    rel_humidity = row['Rel Hum (%)']
    abs_humidity = calculate_abs_humidity(temperature, rel_humidity)
    abs_humidity_list.append(abs_humidity)

3 loops, best of 3: 8.99 s per loop


In [21]:
%%timeit -n 3
abs_humidity_series = df.apply(lambda row:        
    calculate_abs_humidity(row['Temp (°C)'], row['Rel Hum (%)']),         
    axis=1)

3 loops, best of 3: 2.01 s per loop


In [22]:
import numpy as np

In [24]:
%%timeit -n 3
abs_humidity_np = np.vectorize(calculate_abs_humidity)(
    df['Temp (°C)'], 
    df['Rel Hum (%)'])

3 loops, best of 3: 37 ms per loop


In [25]:
def calculate_abs_humidity_np(temperature, relative_humidity):
  """ Calculates absolute humidity from temperature and relative humidity """
  t = temperature
  rh = relative_humidity
  abs_humidity = (6.112 * np.exp((17.67*t)/(t+243.5)) * rh * 2.1674)/(273.15+t)
  return abs_humidity

In [26]:
%%timeit -n 3
calculate_abs_humidity_np(df['Temp (°C)'], df['Rel Hum (%)'])

3 loops, best of 3: 6.72 ms per loop
